### IMPORT LIBRARY

In [1]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validate import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *

### CONNECT TO AZURE SQL

In [2]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

### CONNECT TO SHAREPOINT

In [3]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Internal Hub _ Successful Connection!


### READ FILE

In [4]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Can Tho City
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Quang Ninh Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Hung Yen Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Nghe An Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Da Nang City
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Thuan Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Old
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Khanh Hoa Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Bac Ninh Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/BRVT
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Duong Province
/sites/BIHu

In [5]:
#lấy năm
list_folder_sub1=[]
for i in list_folder:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub1=list_folder_sub1+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Can Tho City/2018
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Can Tho City/2019
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Can Tho City/2020
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2021
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2017
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2020
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2022
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2019
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2018
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Quang Ninh Province/2017
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Quang Ninh Province/2021

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Thua Thien Hue Province/2017
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Thua Thien Hue Province/2022
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Thua Thien Hue Province/2021
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Thua Thien Hue Province/2019
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Dong Nai Province/2017
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Dong Nai Province/2019
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Dong Nai Province/2022
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Dong Nai Province/2018
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Dong Nai Province/2021
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Dong Nai Province/Old
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat 

In [6]:
#lấy quý
list_folder_sub2=[]
for i in list_folder_sub1:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub2=list_folder_sub2+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Can Tho City/2018/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Can Tho City/2019/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Can Tho City/2020/Q3
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2021/Q3
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2021/Q1
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2021/Q4
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2021/Q2
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2017/Q3
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2017/Q4
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Ho Chi Minh City/2017/Q2
/sites/BIHub/Shared Documents/Advisor

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Old/Binh Thuan Province/2021
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Old/Da Nang City/2020
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Old/Thua Thien Hue Province/2020
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Old/Thua Thien Hue Province/2021
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Old/Thua Thien Hue Province/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Khanh Hoa Province/2021/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Khanh Hoa Province/2020/Q4
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Khanh Hoa Province/2020/Q2
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Khanh Hoa Province/2022/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Fla

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Hai Phong City/2017/Q3
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Hai Phong City/2017/Q4
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Hai Phong City/2017/Q2
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Hai Phong City/2017/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Hai Phong City/2022/Q2
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Hai Phong City/2020/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Long An Province/2019/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Long An Province/2022/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Long An Province/2018/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Long An Province/2020/Q2
/sites/BIHu

In [33]:
#List file
#Điền tên tỉnh/thành phố, năm, quý cần import vào 3 biến bên dưới, không edit phần còn lại
Province_name=['Can Tho City', 'Binh Thuan Province']
Year=['2019', '2020']
Quarter=['Q4']
#Không edit từ đây trở xuống
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder_sub2:
    if i.split('/')[7] in Province_name and i.split('/')[8] in Year and i.split('/')[9] in Quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_url = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Can Tho City/2019/Q4/VLTH_CanTho_20191231.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Can Tho City/2019/Q4/Apt_CanTho_20191231.xlsx
Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Thuan Province/2020/Q4/Apt_BinhThuan_20201231.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Thuan Province/2020/Q4/VLTH_BinhThuan_20201231.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Thuan Province/2020/Q4/Retail_BinhThuan_20201231.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Thuan Province/2020/Q4/Hotel_BinhThuan_20201231.xlsx
Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Thuan Province/2019/Q4/Apt_BinhThuan_20191231.xlsx
Files name: /sites/BIHub/Shared D

### GET IMPORT FILE

In [34]:
url = []
for i in list_url:
    sector = i.split('/')[-1].split('_')[0].upper()
    #print(sector)
    if sector=='APARTMENT':
        sector='APT'
    else:
        pass
    if sector not in ('x'):
        url.append(i)
    else:
        pass
url

['/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Can Tho City/2019/Q4/VLTH_CanTho_20191231.xlsx',
 '/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Can Tho City/2019/Q4/Apt_CanTho_20191231.xlsx',
 '/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Thuan Province/2020/Q4/Apt_BinhThuan_20201231.xlsx',
 '/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Thuan Province/2020/Q4/VLTH_BinhThuan_20201231.xlsx',
 '/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Thuan Province/2020/Q4/Retail_BinhThuan_20201231.xlsx',
 '/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Thuan Province/2020/Q4/Hotel_BinhThuan_20201231.xlsx',
 '/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Thuan Province/2019/Q4/Apt_BinhThuan_20191231.xlsx',
 '/sites/BIHub/Shared Documents/Advisory Data/Main sector/Flat file/Binh Thuan Province/2019/Q4/VLTH_BinhThuan_20191231.xlsx',
 '/s

### GET DATA

In [38]:
list_sector = []
columns_that_need_unidecode=['Project_Name','Sub_Project_Name','Developer','Constructor',
                             'Operator','Project_District_Name','Project_City_Name']

df_dict = pd.DataFrame(columns=['File Name', 'Missing values', 'Flag'])

#Create multi empty dataframe for tracking autdit step
name_sector = ['RETAIL', 'SA', 'APT', 'VLTH', 'OFFICE', 'HOTEL']
name_sector = [x.lower() for x in name_sector]
for i in name_sector:
    globals()['df_temp_flat_{}'.format(i)] = pd.DataFrame([])
    globals()['df_flat_{}'.format(i)] = pd.DataFrame([])

    #-------------------------------------------------------
    
for file_url in tqdm(url):
    #Get data
    sp_object = relative_url.split('/')[2].replace('-','')
    data=eval(sp_object).get_file(file_url)
    #-------------------------------------------------------
    data.dropna(axis=0, how='all', thresh=None, subset=None, inplace=True)#remove blank rows  
    data.drop(data.columns[data.columns.str.contains('Unnamed')], axis=1, inplace=True)#remove blank columns
    data.columns = data.columns.str.strip().str.replace(r'\s+', '', regex=True)
    file_name = file_url.split('/')[-1].split('.')[0]
    sector=file_url.split('/')[-1].upper().split('_')[0]
    list_sector.append(sector)
    if sector=='APT':
        sector='APARTMENT'
    elif sector=='SA':
        sector='SERVICED_APARTMENT'
    else:
        pass
    
    #Check date_key
    data = check_date_key(file_url, data)
    
    #Check duplicate
    flag_dup, data, dup=Check_Duplicated(data, 'Sub_Project_Name')
    if flag_dup==1:
        print(colored('Check dupplicate sub_name', 'yellow'))
    else:
        #change np.nan to None
        for i in list(data.columns):
            data[i] = data[i].replace({np.nan: None})
            data[i] = data[i].replace(r'^\s*$', np.nan, regex=True)#Replacing blank values (has white space)
        #Fill up project_name if null:
        data['Project_Name']= np.where(data['Project_Name'].isnull(), data['Sub_Project_Name'], data['Project_Name'])
        
        #-------------------------------------------------------
        #Validation
        #remove unicode
        for i in columns_that_need_unidecode:
            data[i]=Convert_Column(data[i])    
            
        #remove infortmated values
        data=Generate_Additional_Columns(data,url,df_summ_file,BIHub,engine,file_url)
        data = remove_unformated_character(sector, file_name, data) 
        
        #Check dictionary
        
        data, df_dict = check_dictionary(df_dict, file_name, data, 'Project_City_Name', 'City', sector, engine)
        data, df_dict = check_dictionary(df_dict, file_name, data, 'Project_District_Name', 'District', sector, engine)
        data, df_dict = check_dictionary(df_dict, file_name, data, 'Project_Status', 'Status', sector, engine)
        data, df_dict = check_dictionary(df_dict, file_name, data, 'Sub_Project_Type', 'Type', sector, engine)
        data, df_dict = check_dictionary(df_dict, file_name, data, 'Grade', 'Grade', sector, engine)
        '''total_dis_list=[]
        flag_district,data,dis_list=Check_District(sector, data, engine)
        if flag_district==1:
            total_dis_list.append(dis_list)
        else:
            pass
        flag_city,data,dis_list=Check_City(sector, data, engine)
        if flag_city==1:
            total_dis_list.append(dis_list)
        else:
            pass
        #Duyệt qua dictionary Status
        flag_status,data,dis_list=Check_Status(data,engine)
        if flag_status==1:
            total_dis_list.append(dis_list)
        else:
            pass
        #Duyệt qua dictionary Grade và Type
        flag_type,data,dis_list=Check_Type(data,engine,sector)
        if flag_type==1:
            total_dis_list.append(dis_list)
        else:
            pass
        flag_grade,data,dis_list=Check_Grade(data,engine,sector)
        if flag_grade==1:
            total_dis_list.append(dis_list)
        else:
            pass
        #main scripts
        if flag_district==0 and flag_city==0 and flag_grade==0 and flag_type==0 and flag_status==0:'''
        if len(df_dict) != 0:
            print(colored('Missing values in dictionary'),'yellow')
        else:
            print(colored('Validate succesfully','green'))
            #data=Generate_Additional_Columns(data,url,df_summ_file,BIHub,engine,file_url)
            #create tracking audit
            if sector == 'OFFICE':
                df_temp_flat_office = pd.concat([df_temp_flat_office, data], axis=0)
                df_flat_office=tracking_flat_file(df_temp_flat_office, sector)
            elif sector == 'RETAIL':
                df_temp_flat_retail = pd.concat([df_temp_flat_retail, data], axis=0)
                df_flat_retail=tracking_flat_file(df_temp_flat_retail, sector)
            elif sector == 'HOTEL':
                df_temp_flat_hotel = pd.concat([df_temp_flat_hotel, data], axis=0)
                df_flat_hotel=tracking_flat_file(df_temp_flat_hotel, sector)
            elif sector == 'SA' or sector=='SERVICED_APARTMENT':
                df_temp_flat_sa = pd.concat([df_temp_flat_sa, data], axis=0)
                df_flat_sa=tracking_flat_file(df_temp_flat_sa, sector)
            elif sector == 'APT' or sector=='APARTMENT':
                df_temp_flat_apt = pd.concat([df_temp_flat_apt, data], axis=0)
                df_flat_apt=tracking_flat_file(df_temp_flat_apt, sector)
            elif sector == 'VLTH':
                df_temp_flat_vlth = pd.concat([df_temp_flat_vlth, data], axis=0)
                df_flat_vlth=tracking_flat_file(df_temp_flat_vlth, sector)
            else:
                pass
            processed_data, list_column_name_insert_project, flag_key=Check_Project_Key(data,url,relative_url,df_summ_file,BIHub,engine,sector,file_url)
            data=Get_Project_Key(sector, data, processed_data, list_column_name_insert_project, flag_key, engine)
            '''Insert_to_fresh(sector,engine,data,file_url)
        else:
            print(dis_list)'''
    


  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

Validate succesfully


C:\Users\NNgocPhuongUyen\Downloads\Scripts_Import\Scripts_Import\validate.py:561: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data.rename(columns = {'Project_Status':'Current_Status','Import_date':'Import_Date'}, inplace = True)
 11%|█████████▎                                                                          | 1/9 [00:06<00:53,  6.69s/it]

Validate succesfully


C:\Users\NNgocPhuongUyen\Downloads\Scripts_Import\Scripts_Import\validate.py:561: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data.rename(columns = {'Project_Status':'Current_Status','Import_date':'Import_Date'}, inplace = True)
 22%|██████████████████▋                                                                 | 2/9 [00:12<00:44,  6.42s/it]

...

C:\Users\NNgocPhuongUyen\Downloads\Scripts_Import\Scripts_Import\validate.py:561: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data.rename(columns = {'Project_Status':'Current_Status','Import_date':'Import_Date'}, inplace = True)
 78%|█████████████████████████████████████████████████████████████████▎                  | 7/9 [00:43<00:12,  6.32s/it]

Validate succesfully


C:\Users\NNgocPhuongUyen\Downloads\Scripts_Import\Scripts_Import\validate.py:561: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data.rename(columns = {'Project_Status':'Current_Status','Import_date':'Import_Date'}, inplace = True)
 89%|██████████████████████████████████████████████████████████████████████████▋         | 8/9 [00:49<00:06,  6.16s/it]

Column Construction_Status has been added in Hotel_BinhThuan_20191231
Validate succesfully


C:\Users\NNgocPhuongUyen\Downloads\Scripts_Import\Scripts_Import\validate.py:561: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data.rename(columns = {'Project_Status':'Current_Status','Import_date':'Import_Date'}, inplace = True)
100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:55<00:00,  6.22s/it]


### SEND EMAIL

In [39]:
#Process to send email
list_sector = list(set(list_sector))
list_df_flat = [df_flat_office, df_flat_retail, df_flat_hotel, df_flat_sa, df_flat_apt, df_flat_vlth]
df_flat_html = create_content(type_content = 1, type_sector = 1, list_sector = list_sector, list_df_flat = list_df_flat, engine = engine)
df_query_html = create_content(type_content = 2, type_sector = 1, list_sector = list_sector, list_df_flat = list_df_flat, engine = engine)
to_email = ['nngocphuonguyen@savills.com.vn']
run_email(email_type = 1, user_email = to_email, df_flat_html = df_flat_html, df_query_html = df_query_html)

Sent email successfully
